In [ ]:
from HelperFunctions import copy_subfolders_into_another_folder

paths_txt = r'C:\Users\DELL\Graduation Project\Codes and Scripts\validation.txt'

# Set the source and destination folder paths
source_folder = r"E:\Graduation Project\Datasets\Brats21"
destination_folder = r"E:\Graduation Project\Datasets\BraTS Validation set"

copy_subfolders_into_another_folder(paths_txt, source_folder, destination_folder)

# Imports

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage.transform import resize
from tensorflow.keras.utils import plot_model
import pathlib
import imageio
import glob
import PIL
import nibabel as nib
import os
from tkinter import Tcl
import cv2
# import tensorflow_docs.vis.embed as embed
# import tensorflow_addons.layers as tfal
# from keras.initializers import RandomNormal
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input,Conv2D,Conv2DTranspose,LeakyReLU,Activation,Concatenate
from scipy import ndimage
import shutil
import os

from HelperFunctions import preprocess_image_train, generate_images_GIF, predict_image, black_seq_generator
from SynthModels import SqueezeAttention, ResNet, unet_model, Discriminator, old_squeeze, ResNet4

C:\Users\DELL\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


### Reading Cropped BraTS21 examples, change the root_path to you base address

In [1]:
set_data = input('Enter the subset of the data, e.g. 100-200, 200-300   ')

Enter the subset of the data, e.g. 100-200, 200-300   100-200


In [8]:
root_path = r'E:\Graduation Project\Datasets\Brain {} Cropped\{} Niftis Cropped-3D-T Brats'.format(set_data, set_data)  
data_list = sorted(glob.glob(root_path + '/*'))                            #list of paths of the inside files
results_path = r'E:\Graduation Project\Datasets\Synthesized Brats {} BC'.format(set_data)

In [4]:
''' 
Just for 100-200 with ResNet7-30
'''

root_path = r'E:\Graduation Project\Datasets\Brain 200-300 Cropped\200-300 Niftis Cropped-3D-T Brats'  
data_list = sorted(glob.glob(root_path + '/*'))                            #list of paths of the inside files
results_path = r'E:\Graduation Project\Datasets\Synthesized Brats 200-300 BC'

In [3]:
''' 
Just for Validation Set of BraTS
'''

root_path = r'E:\Graduation Project\Datasets\BraTS Validation Brain Cropped'  
data_list = sorted(glob.glob(root_path + '/*'))                            #list of paths of the inside files
results_path = r'E:\Graduation Project\Datasets\Synthesized BraTS Validation Brain Cropped'

In [3]:
''' 
Just for Validation Set of BraTS No Cropped
'''

root_path = r'E:\Graduation Project\Datasets\BraTS Validation set'  
data_list = sorted(glob.glob(root_path + '/*'))                            #list of paths of the inside files
results_path = r'E:\Graduation Project\Datasets\No Crop Synthesized BraTS Validation Brain Cropped'

In [ ]:
for path in data_list:
    path_num = os.path.basename(path)
    new_result_path = os.path.join(results_path, path_num)

    new_T1_path = os.path.join(new_result_path, "images", "GIF T1", "T1_GIF")
    new_T2_path = os.path.join(new_result_path, "images", "GIF T2", "T2_GIF")
    new_Flair_path = os.path.join(new_result_path, "images", "GIF Flair", "Flair_GIF")
    
    if not os.path.exists(new_T1_path):
        os.makedirs(new_T1_path)

    if not os.path.exists(new_T2_path):
        os.makedirs(new_T2_path)

    if not os.path.exists(new_Flair_path):
        os.makedirs(new_Flair_path)

            
    modules_list = sorted(glob.glob(path + '/*'))  
    flair_vol_path = modules_list[0]                      #path of flair volume is always the first ('sorted')
    t1_vol_path = modules_list[2]                         #path of T1 volume is always the third ('sorted')
    t2_vol_path = modules_list[4]                         #path of T2 volume is always the fifth ('sorted')

    flair_nifti = nib.load(flair_vol_path).get_fdata()                    #Flair volume
    T1_nifti = nib.load(t1_vol_path).get_fdata()                          #T1 volume
    T2_nifti = nib.load(t2_vol_path).get_fdata()                          #T2 volume
    

    for i in range(T1_nifti.shape[2]):
        flair_nifti_slice = flair_nifti[:, :, i]
        t1_nifti_slice = T1_nifti[:, :, i]
        t2_nifti_slice = T2_nifti[:, :, i]
        
        flair_rslt_path = f"{new_Flair_path}/{i:03d}.png"
        t1_rslt_path = f"{new_T1_path}/{i:03d}.png"
        t2_rslt_path = f"{new_T2_path}/{i:03d}.png"

        imageio.imwrite(flair_rslt_path, flair_nifti_slice)
        imageio.imwrite(t1_rslt_path, t1_nifti_slice)
        imageio.imwrite(t2_rslt_path, t2_nifti_slice)

In [4]:
# brats_num = input("please enter the brats number  ")
# brats_num = '028'
seq_1 = 'T1'
seq_2 = 'FLAIR'

# Predict

In [5]:
generator_g = SqueezeAttention().model
generator_f = SqueezeAttention().model

discriminator_x = Discriminator().model
discriminator_y = Discriminator().model

generator_g_optimizer = tf.keras.optimizers.Adam(8e-5) 
generator_f_optimizer = tf.keras.optimizers.Adam(8e-5)
discriminator_x_optimizer = tf.keras.optimizers.Adam(8e-5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(8e-5)

In [6]:
print("Generator's parameters = {:,}".format(generator_g.count_params()))
print("Discriminator's parameters = {:,}".format(discriminator_x.count_params()))

Generator's parameters = 10,882,179
Discriminator's parameters = 2,765,569


In [7]:
#TO DO: Generalize Path, add it to arguments
def generate_images_GIF(img_input, model, img_true, mode, order):
    prediction = model(img_input)
    pred_vol = prediction[0, :, :, 0].numpy().copy()
    error = tf.image.ssim(img_true, prediction, max_val=2)
    img_input = np.rot90(img_input[0, :, :, 0], 3)
    img_true = np.rot90(img_true[0, :, :, 0], 3)
    prediction = np.rot90(prediction[0, :, :, 0], 3)
    
    plt.figure(figsize=(10, 6))
    if mode == 1:        
        display_list = [img_input, prediction, img_true]
        title = [f'{seq_1} True', f'{seq_2} predicted', f'{seq_2} True']
      
    else:
        display_list = [img_input, prediction, img_true]
        title = [f'{seq_2} True', f'{seq_1} predicted', f'{seq_1} True']
    
    plots_path_T1_FLAIR = r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF'.format(brats_num, seq_1, seq_2)
    plots_path_FLAIR_T1 = r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF'.format(brats_num, seq_2, seq_1)
    if not os.path.exists(plots_path_T1_FLAIR):
        os.makedirs(plots_path_T1_FLAIR)
    if not os.path.exists(plots_path_FLAIR_T1):
        os.makedirs(plots_path_FLAIR_T1)
        
    
    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i] * 0.5 + 0.5, cmap='gray') 
        plt.axis('off')
        if mode == 1:
            plt.savefig(r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF\{}.png'.format(brats_num, seq_1, seq_2, order)) 
        if mode == 2:
            plt.savefig(r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF\{}.png'.format(brats_num, seq_2, seq_1, order))
    plt.show()
    return error, pred_vol

In [8]:
#TO DO: Generalize Path, add it to arguments
def predict_image(img_input, model, img_true):
    prediction = model(img_input)
    pred_vol = prediction[0, :, :, 0].numpy().copy()
    error = tf.image.ssim(img_true, prediction, max_val=2)
    return error, pred_vol

In [9]:
# checkpoint_path = r"E:\Graduation Project\GIFs and Models\Attention UNET-1"
# checkpoint_path = r"E:\Graduation Project\GIFs and Models\259 CheckPoint 17-2-2023-T1-FLAIR"
# checkpoint_path = r"E:\Graduation Project\GIFs and Models\ResNet-7 7-3-2023 T2-Flair-71"
# checkpoint_path = r"E:\Graduation Project\GIFs and Models\Squeeze Attention U-Net-30"
# checkpoint_path = r"E:\Graduation Project\Graduation Projects GANs Conversion\Trained_Model_epoch_150"
checkpoint_path = r"E:\Graduation Project\GIFs and Models\Squeeze Attention T1-FLAIR-UNET-30"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
#     ckpt.restore(ckpt_manager.checkpoints[30-1])
#     ckpt.restore(ckpt_manager.latest_checkpoint)
#     print(f'Last Check Point: {ckpt_manager.latest_checkpoint} is restored')
    print(f'Last Check Point: {ckpt_manager.latest_checkpoint} is restored')

Last Check Point: E:\Graduation Project\GIFs and Models\Squeeze Attention T1-FLAIR-UNET-30\ckpt-30 is restored


In [10]:
# # Open the input file
# with open("000-100 ResNet7-T2-FLAIR-71.log", "r") as f:
#     lines = f.readlines()

# # Create a new list with every fourth line
# new_lines = lines[::4]

# # Open the output file and write new lines
# with open("modified_log_file.log", "w") as f:
#     f.writelines(new_lines)

In [11]:
# # Open the input file
# with open("000-100 ResNet7-T2-FLAIR-71.log", "r") as f:
#     lines = f.readlines()

# # Remove duplicated lines
# unique_lines = list(set(lines))

# # Open the output file and write unique lines
# with open("unique_log_file.log", "w") as f:
#     f.writelines(unique_lines)

In [19]:
with open('BraTS Validation on SqzAtt T1-T2 30 epochs.log', 'w') as file:
    file.write('')

In [12]:
import logging

# Create a logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create a file handler and set the formatter
file_handler = logging.FileHandler('test smth disaster BraTS Validation on SqzAtt T1-T2 30 epochs.log')
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('Brats %(iteration)s - T1-T2 ssim = %(ssim_score).4f')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

In [10]:
vol_1 = []
vol_2 = []

ssim_1_list = []
ssim_2_list = []

order_1 = 0
order_2 = 0

ssim_score_1 = 0
ssim_score_2 = 0

seq_1 = 'T1'
seq_2 = 'FLAIR'

In [10]:
results_path

'E:\\Graduation Project\\Datasets\\Synthesized BraTS Validation Brain Cropped'

# Copy from root to result

In [12]:
# source_folder = '/path/to/source/folder'

# results_path = r'E:\Graduation Project\Datasets\Synthesized Brats 200-300 BC'
results_data_list = sorted(glob.glob(results_path + '/*')) 

T1_syn_result_vol_path = r'E:\Graduation Project\Datasets\SqzAtt(30 ep) - T1 Synth from FLAIR - BraTS Validation - Brain Cropped'
# T2_syn_result_vol_path = r'E:\Graduation Project\Datasets\ResNet4(259 ep) - T1 Synth from FLAIR - BraTS Validation - Brain Cropped'
FLAIR_syn_result_vol_path = r'E:\Graduation Project\Datasets\SqzAtt(30 ep) - FLAIR Synth from T1 - BraTS Validation - Brain Cropped'

# Get a list of all the subfolders in the source folder
subfolders = [f.path for f in os.scandir(root_path) if f.is_dir()]

# Copy each subfolder to the two destination folders
for folder in subfolders:
    shutil.copytree(folder, os.path.join(T1_syn_result_vol_path, os.path.basename(folder)))
#     shutil.copytree(folder, os.path.join(T2_syn_result_vol_path, os.path.basename(folder)))
    shutil.copytree(folder, os.path.join(FLAIR_syn_result_vol_path, os.path.basename(folder)))

T1_syn_result_vols = sorted(glob.glob(T1_syn_result_vol_path + '/*'))
# T2_syn_result_vols = sorted(glob.glob(T2_syn_result_vol_path + '/*'))
FLAIR_syn_result_vols = sorted(glob.glob(FLAIR_syn_result_vol_path + '/*'))

In [ ]:
results_path = r'E:\Graduation Project\Datasets\Synthesized Brats 200-300 BC'
results_data_list = sorted(glob.glob(results_path + '/*')) 

# T1_syn_result_vol_path = r'E:\Graduation Project\Datasets\ResNet7-71 FLAIR Synthesized Brats 000-100 Vols'
T2_syn_result_vol_path = r'E:\Graduation Project\Datasets\ResNet7-71 T2 Synthesized Brats 200-300 Vols'
FLAIR_syn_result_vol_path = r'E:\Graduation Project\Datasets\ResNet7-71 FLAIR Synthesized Brats 200-300 Vols'

# T1_syn_result_vols = sorted(glob.glob(T1_syn_result_vol_path + '/*'))
T2_syn_result_vols = sorted(glob.glob(T2_syn_result_vol_path + '/*'))
FLAIR_syn_result_vols = sorted(glob.glob(FLAIR_syn_result_vol_path + '/*'))

# Delete T1

In [13]:
del_root_path = 'E:/Graduation Project/Datasets/ResNet259 T1 Synthesized Brats 000-100 Vols'

for dirpath, dirnames, filenames in os.walk(T1_syn_result_vol_path):
    for filename in filenames:
        if filename.endswith('t1.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            os.remove(file_path)
#             print(f"{file_path} has been deleted.")

# Delete T2

In [14]:
del_root_path = 'E:/Graduation Project/Datasets/ResNet7-71 T2 Synthesized Brats 200-300 Vols'

for dirpath, dirnames, filenames in os.walk(T2_syn_result_vol_path):
    for filename in filenames:
        if filename.endswith('t2.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            os.remove(file_path)
#             print(f"{file_path} has been deleted.")

# Delete FLAIR

In [14]:
del_root_path = 'E:/Graduation Project/Datasets/ResNet7-71 FLAIR Synthesized Brats 200-300 Vols'

for dirpath, dirnames, filenames in os.walk(FLAIR_syn_result_vol_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            os.remove(file_path)
#             print(f"{file_path} has been deleted.")

In [16]:
results_data_list = results_data_list[103:]

In [15]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
ssim1 = []
ssim2 = []

for i, path in enumerate(results_data_list):
#     if kkk == 0:
#         kkk = 1
#         continue
    T1_path = os.path.join(path, "images", "GIF T1")
#     T2_path = os.path.join(path, "images", "GIF T2")
    FLAIR_path = os.path.join(path, "images", "GIF Flair")

    dep = len(glob.glob(T1_path + '/**/*'))
    
    GIF_T1 = tf.keras.preprocessing.image_dataset_from_directory(
                                  T1_path,
                                  seed=123,
                                  image_size=(256, 256),
                                  batch_size=1,
                                  shuffle = False)

#     GIF_T2 = tf.keras.preprocessing.image_dataset_from_directory(
#                                   T2_path,
#                                   seed=123,
#                                   image_size=(256, 256),
#                                   batch_size=1,
#                                   shuffle = False)
    
    GIF_FLAIR = tf.keras.preprocessing.image_dataset_from_directory(
                                  FLAIR_path,
                                  seed=123,
                                  image_size=(256, 256),
                                  batch_size=1,
                                  shuffle = False)
                            

    GIF_T1 = GIF_T1.cache().prefetch(buffer_size=AUTOTUNE)
#     GIF_T2 = GIF_T2.cache().prefetch(buffer_size=AUTOTUNE)
    GIF_FLAIR = GIF_FLAIR.cache().prefetch(buffer_size=AUTOTUNE)

    GIF_T1 = GIF_T1.map(lambda x, _: (preprocess_image_train(x)))
#     GIF_T2 = GIF_T2.map(lambda x, _: (preprocess_image_train(x)))
    GIF_FLAIR = GIF_FLAIR.map(lambda x, _: (preprocess_image_train(x)))


    for image_x, image_y in tf.data.Dataset.zip((GIF_T1, GIF_FLAIR)):
        ssim_1, img_1 = predict_image(image_x, generator_g, image_y)
        ssim_score_1 = ssim_score_1 + ssim_1
        ssim_2_list.append(ssim_1)
        vol_2.append(img_1)

    for image_x, image_y in tf.data.Dataset.zip((GIF_T1, GIF_FLAIR)):
        ssim_2, img_2 = predict_image(image_y, generator_f, image_x)
        ssim_score_2 = ssim_score_2 + ssim_2
        ssim_1_list.append(ssim_2)
        vol_1.append(img_2)
    
    ssim_score_1, ssim_score_2 = ssim_score_1/dep , ssim_score_2/dep
#     logger.info('', extra={'iteration': i+200, 'ssim_score': ssim_score_1.numpy()[0]})
#     logger.info('', extra={'iteration': i+200, 'ssim_score': ssim_score_2.numpy()[0]})


    original_vol_path = sorted(glob.glob(data_list[i] + '/*'))[0]
    original_vol = nib.load(original_vol_path)
    original_shape = original_vol.shape


    vol_1 = np.array(vol_1).transpose(1, 2, 0)
    vol_2 = np.array(vol_2).transpose(1, 2, 0)

    vol_1 = ndimage.zoom(vol_1, (original_shape[0]/vol_1.shape[0],
                                 original_shape[1]/vol_1.shape[1],
                                 original_shape[2]/vol_1.shape[2]), order=0)

    vol_2 = ndimage.zoom(vol_2, (original_shape[0]/vol_2.shape[0],
                                 original_shape[1]/vol_2.shape[1],
                                 original_shape[2]/vol_2.shape[2]), order=0)


    v1 = nib.Nifti1Image(np.array(vol_1), original_vol.affine)            # to save this 3D (ndarry) numpy
    v2 = nib.Nifti1Image(np.array(vol_2), original_vol.affine)            # to save this 3D (ndarry) numpy

    T1_name = os.path.basename(sorted(glob.glob(data_list[i] + '/*'))[2])
#     T2_name = os.path.basename(sorted(glob.glob(data_list[i] + '/*'))[4])
    FLAIR_name = os.path.basename(sorted(glob.glob(data_list[i] + '/*'))[0])

    T1_res_path = os.path.join(T1_syn_result_vols[i], T1_name)
#     T2_res_path = os.path.join(T2_syn_result_vols[i], T2_name)
    FLAIR_res_path = os.path.join(FLAIR_syn_result_vols[i], FLAIR_name)


    # Save the Nifti image to a file
    nib.save(v1, T1_res_path)
#     nib.save(v2, T2_res_path)
    nib.save(v2, FLAIR_res_path)

    
    vol_1 = []
    vol_2 = []
    
    ssim1.append(ssim_score_1)
    ssim2.append(ssim_score_2)

    ssim_score_1 = 0
    ssim_score_2 = 0
    print(f"Volume #{i} is done")

Found 146 files belonging to 1 classes.
Found 146 files belonging to 1 classes.
Volume #0 is done
Found 131 files belonging to 1 classes.
Found 131 files belonging to 1 classes.
Volume #1 is done
Found 138 files belonging to 1 classes.
Found 138 files belonging to 1 classes.
Volume #2 is done
Found 139 files belonging to 1 classes.
Found 139 files belonging to 1 classes.
Volume #3 is done
Found 135 files belonging to 1 classes.
Found 135 files belonging to 1 classes.
Volume #4 is done
Found 146 files belonging to 1 classes.
Found 146 files belonging to 1 classes.
Volume #5 is done
Found 129 files belonging to 1 classes.
Found 129 files belonging to 1 classes.
Volume #6 is done
Found 144 files belonging to 1 classes.
Found 144 files belonging to 1 classes.
Volume #7 is done
Found 126 files belonging to 1 classes.
Found 126 files belonging to 1 classes.
Volume #8 is done
Found 142 files belonging to 1 classes.
Found 142 files belonging to 1 classes.
Volume #9 is done
Found 141 files belo

Volume #83 is done
Found 140 files belonging to 1 classes.
Found 140 files belonging to 1 classes.
Volume #84 is done
Found 130 files belonging to 1 classes.
Found 130 files belonging to 1 classes.
Volume #85 is done
Found 145 files belonging to 1 classes.
Found 145 files belonging to 1 classes.
Volume #86 is done
Found 141 files belonging to 1 classes.
Found 141 files belonging to 1 classes.
Volume #87 is done
Found 146 files belonging to 1 classes.
Found 146 files belonging to 1 classes.
Volume #88 is done
Found 123 files belonging to 1 classes.
Found 123 files belonging to 1 classes.
Volume #89 is done
Found 138 files belonging to 1 classes.
Found 138 files belonging to 1 classes.
Volume #90 is done
Found 136 files belonging to 1 classes.
Found 136 files belonging to 1 classes.
Volume #91 is done
Found 146 files belonging to 1 classes.
Found 146 files belonging to 1 classes.
Volume #92 is done
Found 133 files belonging to 1 classes.
Found 133 files belonging to 1 classes.
Volume #93

Volume #166 is done
Found 134 files belonging to 1 classes.
Found 134 files belonging to 1 classes.
Volume #167 is done
Found 139 files belonging to 1 classes.
Found 139 files belonging to 1 classes.
Volume #168 is done
Found 135 files belonging to 1 classes.
Found 135 files belonging to 1 classes.
Volume #169 is done
Found 141 files belonging to 1 classes.
Found 141 files belonging to 1 classes.
Volume #170 is done
Found 131 files belonging to 1 classes.
Found 131 files belonging to 1 classes.
Volume #171 is done
Found 152 files belonging to 1 classes.
Found 152 files belonging to 1 classes.
Volume #172 is done
Found 139 files belonging to 1 classes.
Found 139 files belonging to 1 classes.
Volume #173 is done
Found 134 files belonging to 1 classes.
Found 134 files belonging to 1 classes.
Volume #174 is done
Found 146 files belonging to 1 classes.
Found 146 files belonging to 1 classes.
Volume #175 is done
Found 139 files belonging to 1 classes.
Found 139 files belonging to 1 classes.


Volume #248 is done
Found 127 files belonging to 1 classes.
Found 127 files belonging to 1 classes.
Volume #249 is done
Found 135 files belonging to 1 classes.
Found 135 files belonging to 1 classes.
Volume #250 is done


In [16]:
import pandas as pd

# Create a DataFrame from the lists
df = pd.DataFrame({
    "From T1 to FLAIR": np.array(ssim2)[:, 0],
    'From FLAIR to T1': np.array(ssim1)[:, 0]
})

# Save the DataFrame to a CSV file
df.to_csv('T1-FLAIR SqzAtt-30 SSIM.csv', index=False)

In [25]:
T2_res_path = os.path.join(T2_syn_result_vols[i], T2_name)
FLAIR_res_path = os.path.join(FLAIR_syn_result_vols[i], FLAIR_name)


# Save the Nifti image to a file
nib.save(v1, FLAIR_res_path)
#     nib.save(v2, T2_res_path)
nib.save(v2, T2_res_path)


vol_1 = []
vol_2 = []

ssim_score_1 = 0
ssim_score_2 = 0
print(f"Volume #{i} is done")

Volume #0 is done


In [31]:
np.shape(vol_1)

(135, 256, 256)

In [28]:
ssim_score_1, ssim_score_2 = ssim_score_1/dep , ssim_score_2/dep


print("ssim_score for first Conversion = {:.4f} ".format(ssim_score_1.numpy()[0]))
print("ssim_score for second Conversion = {:.4f} ".format(ssim_score_2.numpy()[0]))

ssim_score for first Conversion = 0.6095 
ssim_score for second Conversion = 0.6937 


In [29]:
original_vol_path = sorted(glob.glob(root_path + '/*'))[0]
original_vol = nib.load(original_vol_path)
x = original_vol.get_fdata()
print(x.min(), x.max())



original_vol = nib.load(r'E:\Graduation Project\GIFs and Models\Brats 006\Testing Synthesis With Segmentation on BraTS #006\FLAIR.nii.gz')
x = original_vol.get_fdata()
print(x.min(), x.max())

0.0 3724.0
-1.0 0.9064252972602844


In [30]:
original_vol_path = sorted(glob.glob(root_path + '/*'))[0]
original_vol = nib.load(original_vol_path)
original_shape = original_vol.shape

result_vol_path = r'E:\Graduation Project\GIFs and Models\Brats {}\Synthesized Volumes'.format(brats_num)
if not os.path.exists(result_vol_path):
    os.makedirs(result_vol_path)

vol_1 = np.array(vol_1).transpose(1, 2, 0)
vol_2 = np.array(vol_2).transpose(1, 2, 0)


vol_1 = ndimage.zoom(vol_1, (original_shape[0]/vol_1.shape[0],
                                             original_shape[1]/vol_1.shape[1],
                                             original_shape[2]/vol_1.shape[2]), order=0)

vol_2 = ndimage.zoom(vol_2, (original_shape[0]/vol_2.shape[0],
                                             original_shape[1]/vol_2.shape[1],
                                             original_shape[2]/vol_2.shape[2]), order=0)

    
v1 = nib.Nifti1Image(np.array(vol_1), original_vol.affine)            # to save this 3D (ndarry) numpy
v2 = nib.Nifti1Image(np.array(vol_2), original_vol.affine)            # to save this 3D (ndarry) numpy


# Save the Nifti image to a file
nib.save(v1, result_vol_path + '/' + 'T1.nii.gz')
nib.save(v2, result_vol_path + '/' + 'T2.nii.gz')

In [ ]:
anim_file = r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}.gif'.format(brats_num, seq_1, seq_2)

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob(r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF\*.png'.format(brats_num, seq_1, seq_2))
  filenames = sorted(filenames)
  filenames = list(Tcl().call('lsort', '-dict', filenames))
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

embed.embed_file(anim_file)

In [ ]:
anim_file = r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}.gif'.format(brats_num, seq_2, seq_1)

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob(r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF\*.png'.format(brats_num, seq_2, seq_1))
  filenames = sorted(filenames)
  filenames = list(Tcl().call('lsort', '-dict', filenames))
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

embed.embed_file(anim_file)

In [ ]:
k = 0
for i, path in enumerate(results_data_list):
#     if kkk == 0:
#         kkk = 1
#         continue
    T1_path = os.path.join(path, "images", "GIF T1")
    T2_path = os.path.join(path, "images", "GIF T2")
    FLAIR_path = os.path.join(path, "images", "GIF Flair")

    dep = len(glob.glob(T1_path + '/**/*'))
    
    GIF_T1 = tf.keras.preprocessing.image_dataset_from_directory(
                                  T1_path,
                                  seed=123,
                                  image_size=(256, 256),
                                  batch_size=1,
                                  shuffle = False)

    GIF_T2 = tf.keras.preprocessing.image_dataset_from_directory(
                                  T2_path,
                                  seed=123,
                                  image_size=(256, 256),
                                  batch_size=1,
                                  shuffle = False)


    GIF_T1 = GIF_T1.cache().prefetch(buffer_size=AUTOTUNE)
    GIF_T2 = GIF_T2.cache().prefetch(buffer_size=AUTOTUNE)

    GIF_T1 = GIF_T1.map(lambda x, _: (preprocess_image_train(x)))
    GIF_T2 = GIF_T2.map(lambda x, _: (preprocess_image_train(x)))


    for image_x, image_y in tf.data.Dataset.zip((GIF_T1, GIF_T2)):
#         ssim_1, img_1 = predict_image_and_calc_loss(image_x, generator_f, image_y)
#         ssim_score_1 = ssim_score_1 + ssim_1
#         ssim_1_list.append(ssim_1)
#         vol_1.append(img_1)
        prediction = generator_g(image_x)
        print(image_x.numpy().mean(), prediction.numpy().mean())
#         print(image_x.numpy().min(), prediction.numpy().min())
#         plt.imshow(image_y.numpy()[0])
        k = k + 1
        break
    if k == 200:
        break
#     break

In [ ]:
plt.imshow(image_y.numpy()[0])

In [ ]:
for i, path in enumerate(results_data_list):
    T1_path = os.path.join(path, "images", "GIF T1")
    T2_path = os.path.join(path, "images", "GIF T2")
    FLAIR_path = os.path.join(path, "images", "GIF Flair")

    print(len(glob.glob(T1_path + '/**/*')))
    